#### konlpy를 사용한 형태소분리와 Word2Vec를 사용한 벡터변환

In [2402]:
# # 한국어 전처리를 위한
# !pip install konlpy
# !pip install Twitter
# !pip install tqdm
# !pip install gensim
# !pip install --upgrade jupyter ipywidgets

In [2403]:
import os # 운영 체제 관련 작업을 수행하기 위한 Python 표준 라이브러리 중 하나 -> 자바 환경 변수를 설정하기 위해 사용
import pandas as pd # 데이터 조작 및 분석을 위한 파이썬 라이브러리 -> 데이터를 데이터프레임으로 로드하고 조작하는 데 사용
import numpy as np # 배열 및 수치 연산을 수행하기 위한 라이브러리 -> 다차원 배열 및 수학적 함수를 다루는 데 사용
from konlpy.tag import Okt # Konlpy 라이브러리에서 제공하는 한국어 형태소 분석기 중 하나 ->  Okt를 초기화하는 데 사용
from gensim.models.word2vec import Word2Vec # Gensim 라이브러리에서 제공하는 Word2Vec 모델을 생성하기 위한 클래스 -> 단어를 고차원 벡터로 표현하여 단어 간 유사성 및 의미 관계를 파악
import random
from sklearn.metrics.pairwise import cosine_similarity

In [2404]:
# 본인 컴퓨터와 자바 설치 경로 설정
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"

In [2405]:
# 데이터를 데이터프레임으로 로드
data = pd.read_csv('../cleaned_data.csv', encoding='utf-8')# 데이터 파일 경로
print(data)

       SEQ_NO  ISBN_THIRTEEN_NO VLM_NM                              TITLE_NM  \
0     6352228     9791156759270    NaN   너에게 목소리를 보낼게  달빛천사 성우 이용신의 첫 번째 에세이   
1     6352229     9791168120877    NaN  일기에도 거짓말을 쓰는 사람  99년생 시인의 자의식 과잉 에세이   
2     6352230     9791168120839    NaN            본격 한중일 세계사 12  임오군란과 통킹 위기   
3     6352231     9791168120846    NaN   즉시 기분을 바꿔드립니다  신기하게 마음이 편해지는 응급 처방전   
4     6352232     9791168120747    NaN               오늘도 리추얼  음악 나에게 선물하는 시간   
...       ...               ...    ...                                   ...   
1997  6354246     9791164670871    NaN                          학교 서클대화가 필요해   
1998  6354247     9788994229003    NaN                      People Make City   
1999  6354248     9788994027203    NaN                                 동방해경표   
2000  6354249     9791156759126    NaN                      큰글자도서 그렇다면 정상입니다   
2001  6354250     9788988963258    NaN                                 그 사람들   

                  AUTHR_NM PUBLISHER_NM

In [2406]:
# 데이터프레임에서 'BOOK_INTRCN_CN' 열의 값이 비어있지 않은 행만 선택
data = data[data['BOOK_INTRCN_CN'].notna()]

In [2407]:
# 데이터프레임 재색인
data = data.reset_index(drop=True)

In [2408]:
# 학습 데이터로부터 Word2Vec 모델 생성
sentences = [document.split() for document in data['BOOK_INTRCN_CN']]
model_train = Word2Vec(sentences, vector_size=100, window=5, min_count=10, sg=1)

In [2409]:
# 형태소 분리를 위한 Konlpy 객체 초기화
twitter = Okt()

In [2410]:
# 데이터프레임에서 무작위로 한 줄거리를 선택
random_index = random.randint(0, len(data) - 1)
selected_document = data.loc[random_index, 'BOOK_INTRCN_CN']

In [2411]:
# 명사만 추출
filtered_tokens = twitter.nouns(selected_document)
print(filtered_tokens)

['월요일', '조회', '시간', '교장', '선생님', '전학생', '소개', '그', '전학생', '바로', '듯', '근육', '고릴라', '고릴라', '우리', '학교', '전학', '동물원', '고릴라', '반', '생활', '동준', '아이']


In [2412]:
def get_doc_vector(tokens, model):
    doc_vector = np.zeros(100)  # Word2Vec 모델에서 사용한 차원 수 (100 차원)로 맞게 수정
    for token in tokens:
        if token in model.wv:
            doc_vector += model.wv[token]

    num_tokens = len(tokens)
    if num_tokens > 0:
        doc_vector /= num_tokens

    return doc_vector

In [2413]:
# Word2Vec 모델을 사용하여 벡터 추출
selected_doc_vector = get_doc_vector(filtered_tokens, model_train)


In [2414]:
# 모든 다른 줄거리와의 유사도 계산
similarities = []
for i, document_tokens in enumerate(sentences):
    if i != random_index:  # 선택한 줄거리 자체와는 비교하지 않음
        filtered_tokens = twitter.nouns(data.loc[i, 'BOOK_INTRCN_CN'])
        doc_vector = get_doc_vector(filtered_tokens, model_train)
        similarity = cosine_similarity([selected_doc_vector], [doc_vector])[0][0]
        similarities.append(similarity)

In [2415]:
print(similarities)

[0.9984408259125792, 0.9985544112359493, 0.9985455460657964, 0.0, 0.999312509454993, 0.998190150410042, 0.9984349578796268, 0.9984230147612508, 0.9992570941377917, 0.9986515318638634, 0.9983733653755383, 0.9988476298977178, 0.9984211223808418, 0.0, 0.9987993953898708, 0.9995650992294224, 0.9987927097623823, 0.0, 0.9990459432344787, 0.9972681073202391, 0.9993874389302895, 0.998607970199474, 0.9979359843042022, 0.9991173595468332, 0.9976579007182821, 0.0, 0.999078268842319, 0.9974393992740421, 0.9983954048113617, 0.9972681073202392, 0.9992637876640982, 0.9965385490430391, 0.9980191097738875, 0.9982828711723133, 0.9985626499093144, 0.999252386365322, 0.998690815198571, 0.9978104385870693, 0.9986615877920982, 0.0, 0.9985626499093143, 0.9970129098408634, 0.9985601039234171, 0.0, 0.9983954048113617, 0.9981330100830715, 0.9979590877225315, 0.9970062898317323, 0.9991214944099547, 0.9974737558210575, 0.9987723738888088, 0.9982828711723133, 0.0, 0.9991372661168841, 0.9973928131767328, 0.0, 0.998

In [2416]:
# 가장 유사한 N개의 줄거리 추천
N = 5  # 원하는 추천 결과 개수
sorted_indices = np.argsort(similarities)[::-1]  # 유사도가 높은 순으로 정렬된 인덱스

# 자기 자신과 같은 인덱스는 제외하고 가장 유사한 N개 선택
top_indices = [i for i in sorted_indices if i != random_index][:N]

In [2417]:
print(f"선택한 줄거리: {selected_document}")
for i, similar_document_index in enumerate(top_indices):
    similarity = similarities[similar_document_index]
    similar_title = data.loc[similar_document_index, 'TITLE_NM']
    similar_doc = data.loc[similar_document_index, 'BOOK_INTRCN_CN']
    print(f"제목 #{i + 1}: {similar_title}")
    print(f"줄거리: {similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

선택한 줄거리: 월요일 조회 시간 교장 선생님이 특별한 전학생을 소개한다 그 전학생은 바로 터질 듯한 근육을 가진 고릴라이다 엥 고릴라가 우리 학교로 전학을 온다고 동물원에 있어야 할 고릴라와 한 반에서 생활하게 된 동준이와 아이들은 어리둥절하다
제목 #1: 기도 전략 자료집
줄거리: 응답을 받기 위한 기도 지침서 응답이 없는 고장난 기도를 성경적 기도로 무장시켜 성령 충만한 기도 생활을 하며 예수 그리스도께서 그분의 삶을 통해 이미 우리에게 주신 모든 풍성한 능력의 삶을 살게 하는 기도 전략을 담았다
유사도: 0.9996

제목 #2: 아홉수 우리들 14 세트  전4권  완결
줄거리: 밝고 긍정적인 성격의 봉우리는 미대를 나왔지만 작은 출판사의 계약직 편집 디자이너가 되고 어려운 집안 환경 속에서도 악착같이 살아온 시크녀 차우리는 저가 항공사 승무원이 그리고 내성적이지만 성실한 김우리는 스펙 갑인 부모님과 오빠와는 달리 자존감 낮은 공시생으로 생활한다
유사도: 0.9996

제목 #3: 사이 떡볶이
줄거리: 우리 아이들이 생활 속에서 겪고 있는 이성 친구에 대한 감정과 고민 좌절 등을 사실적으로 보여 준다 특히 좋은 이성 친구는 외모가 멋진 친구가 아니라 마음씨가 착하고 자신과 여러 가지로 잘 맞는 친구라는 것을 느끼게 한다
유사도: 0.9996

제목 #4: 똑똑한 하루 계산 5A  혼자 공부하는 4주 완성 연산서 5학년 수준
줄거리: 혼자 공부하는 4주 완성 연산서다 반드시 알아야 할 개념을 만화로 쉽게 이해할 수 있으며 계산 원리와 방법이 한눈에 쏙 들어온다 기초집중연습 코너에서 생활 속 계산 연습과 문장 읽고 식 세우는 연습도 할 수 있다 한 주에 배운 내용을 테스트로 마무리하고 창의융합코딩 문제도 도전할 수 있다
유사도: 0.9994

제목 #5: 똑똑한 하루 계산 6A  혼자 공부하는 4주 완성 연산서 6학년 수준
줄거리: 혼자 공부하는 4주 완성 연산서다 반드시 알아야 할 개념을 만화로 쉽게 이해할 수 있으며 계산 원리와 방법이 한눈에 쏙 들어온